In [36]:
import numpy as np
import pandas as pd
import time
from sklearn.svm import SVC,LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import sklearn.metrics

def read(readdata):
    data = np.array(readdata);
    data = np.delete(data, 0, 0)
    data = np.swapaxes(data,0,1)
    for i in range(0, np.shape(data)[0]):
        data[i,:] = data[i,:] - np.min(data[i,:])
        data[i,:] = (data[i,:]/np.ndarray.max(data[i,:]))*2 - 1
    # Need to seperate the classification dimension:
    classIndex = np.shape(data)[0] - 1
    classification = data[classIndex]
    data = np.delete(data, classIndex, axis=0)
    data = data.astype(float)
    d = np.shape(data)[0]
    nPoints = np.shape(data)[1]
    return data,classification,d,nPoints

readdata = pd.read_csv("/Users/karanmukhi/Desktop/TheGrandTour/Datasets/wineData.txt", sep=",", header=None);
(data, classification, d, nPoints) = read(readdata)
labels = np.array([-1,0,1])


In [2]:
#VARIABLES
stepSize = 0.000000000001
nSteps = 100

def getAlpha(d):
    """
    NEEDS IMPLEMENTATION
    Should produce 1xd(d-1)/2 array of position in grand tour.

    """
    p = d*(d-1)/2     
    primeList = []
    count = 1
    while len(primeList) < p:
        count += 1
        primeBool = False
        for i in range(2, count - 1):
            if count % i == 0:
                primeBool = True
        if primeBool == False:
            irrational = (np.sqrt(count)%1)
            primeList.append(irrational)
            
    primeList = np.asarray(primeList)
    primeList = primeList.dot(stepSize)
    """
    Irrational number generation using exponentials, not being used
    p = int(d*(d-1)/2)
    alpha = np.zeros(p) #alpha(t) parameters defining grand tour in G2,d

    for i in range(0,p):
        alpha[i] = (np.exp(i) % 1) * 2 * np.pi
        
    alpha = alpha.dot(0.001)
    """
    
    return primeList


def getAngles(alpha,d):
    """""
    Inputs: 
    alpha = 1xd(d-1)/2 array defining position on grand tour
    d = dimensions of data
    Outputs a dxd array of angles required for the transformation
    """
    theta = np.zeros((d,d));
    i = 0;
    k = 0;
    
    while i < d-1:
        j = i + 1;
        
        while j < d:
            theta[i][j] = alpha[k];
            j += 1;
            k += 1;
    
        i+= 1;
        
    return theta;


def RotationMatrix(i, j, d, theta):
    """
    Inputs:
    i = first indicie of rotating plane
    j = second indicie of rotating plane
    d = dimensions of data
    theta = dxd array of angle of rotation of rotating plane

    Outputs a rotating matrix to rotate plane of ixj plane by theta_ij
    """
    R = np.identity(d)
    R[i,i] = np.cos(theta)
    R[i,j] = -1*np.sin(theta)
    R[j,i] = np.sin(theta)
    R[j,j] = np.cos(theta)
    return R


def BetaFn(d, theta):
    """
    Inputs:
    d = dimensions of data
    theta = dxd array of angle of rotation ixj plane

    Outputs the full matrix transformation for all rotations
    """
    b = RotationMatrix(1, 2, d, theta[1,2])
    i = 1
    j = 2
    for i in range(d):
        for j in range(d):
            if j <= i:
                continue
            if i==1 and j==2:
                continue
            b = np.matmul(b, RotationMatrix(i, j, d, theta[i,j]))
            
    return b


def GrandTour(data, nSteps):
    """
    Inputs:
    data = array of data points, dimensions x npoints
    Outputs a 3D array number of points x t x dimensions, where t
    the time step at that point in the tour
    """

    d = np.shape(data)[0] #dimensions of data
    nPoints = np.shape(data)[1] #number of data points
    tData = np.zeros((nSteps,d,nPoints)) #initialise 3d matrix to store stransforemd data at each timestep
    tBeta = np.zeros((nSteps,d,d))
    Alpha = getAlpha(d)

    
    for t in range(0, nSteps):
        
        
        alpha = Alpha.dot(t)
        theta = getAngles(alpha, d)
        b = BetaFn(d, theta)
        a = np.matmul(b, data)
        tData[t,:,:] = a
        tBeta[t,:,:] = b
        
    return tData


In [4]:
def costfn(a):
    xData = a[0]
    yData = a[1]
    nData = np.vstack((xData, yData)).T
    clf = OneVsRestClassifier(SVC(kernel='linear'), n_jobs=-1)
    clf.fit(nData, classification)
    p = clf.decision_function(nData)
    score = sklearn.metrics.hinge_loss(classification,p,labels)
    return score


In [42]:
def iterator(X, G, tau):
    """
    Computes Y(tau) or X^{t+1} given X^t and G
    """
    I = np.identity(np.shape(X)[0])
    if np.shape(X)[1] > 0.5 * np.shape(X)[0]:
        W = G * X.T - X * G.T
        term = (I + (tau / 2) * W)
        Y = term.I * term * X
    else:
        
        U = np.concatenate((G, X), axis=1)
        V = np.concatenate((X, -G), axis=1).T
        Y = X - tau * U * (I + (tau / 2) * (V.T * U)).I * V.T * X
    return Y

def derivative(X, costfn, eps=0.0000001):
    G = np.zeros((np.shape(X)[0], np.shape(X)[1]))
    for i in range(np.shape(X)[0]):
        for j in range(np.shape(X)[1]):
            A = X
            A[i, j] = X[i, j] + eps
            G = (costfn(A) - costfn(X)) / eps
    return G

def hingeLoss(a):
    xData = a[0]
    yData = a[1]
    nData = np.vstack((xData, yData)).T
    clf = OneVsRestClassifier(SVC(kernel='linear'), n_jobs=-1)
    clf.fit(nData, classification)
    p = clf.decision_function(nData)
    score = sklearn.metrics.hinge_loss(classification,p,labels)
    return score



In [40]:
xData = a[0]
yData = a[1]
nData = np.vstack((xData, yData)).T

In [41]:
np.shape(nData)

(177, 2)

In [13]:
np.shape(a)

(13, 177)

In [29]:
p = 2
b = a[:p,:]
np.shape(b.T)

(177, 2)

In [26]:
xData = a[0]
yData = a[1]
nData = np.vstack((xData, yData)).T

In [27]:
np.shape(nData)

(177, 2)

In [30]:
np.shape(data)

(13, 177)